# Class Weight Compution

In [11]:
import math
import sys
import os

import matplotlib.pyplot as plt
import numpy as np
    
%matplotlib inline
%load_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True) # don't use scientific notation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
export_ds = '/media/berlukas/Data2/datasets/nuscenes/processed/'
# export_ds = '/media/scratch/berlukas/nuscenes'

# sem_clouds_filename = f"{export_ds}/sem_bkp/sem_classes_gt1.npy"
sem_clouds_filename = f"{export_ds}/sem_clouds2.npy"

print(f"Loading sem clouds from {sem_clouds_filename}")
sem_cloud_features = np.load(sem_clouds_filename).astype(int)
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading sem clouds from /media/berlukas/Data2/datasets/nuscenes/processed//sem_clouds2.npy
Shape of sem clouds is (11452, 3, 200, 200)


In [2]:
export_ds = '/media/scratch/berlukas/nuscenes/training'
samples = os.listdir(export_ds)
sem_cloud_features = None
for sample in samples:
    sem_clouds_filename = f"{export_ds}/{sample}"    
    print(f"Loading from sem clouds from {sem_clouds_filename}")
    if sem_cloud_features is None:
        sem_cloud_features = np.load(sem_clouds_filename).astype(int)
    else:
        features = np.load(sem_clouds_filename).astype(int)
        sem_cloud_features = np.concatenate((sem_cloud_features, features))
print(f"Shape of sem clouds is {sem_cloud_features.shape}")

Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_kitti_00_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_nuscenes5000_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_waymo_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_kitti_02_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_nuscenes_01_bw50.npy
Loading from sem clouds from /media/scratch/berlukas/nuscenes/training/sem_clouds_kitti_05_bw50.npy
Shape of sem clouds is (19562, 2, 100, 100)


In [4]:
n_classes = 6
class_counts = [0] * n_classes
for cl in range(0, n_classes): 
    count = np.count_nonzero(sem_cloud_features[:,1,:,:] == cl)
    print(f'Computing occurrences for {cl}: {count}')
    class_counts[cl] = count

Computing occurrences for 0: 172123974
Computing occurrences for 1: 1801972
Computing occurrences for 2: 75612
Computing occurrences for 3: 12203294
Computing occurrences for 4: 4354036
Computing occurrences for 5: 4257417


In [12]:
print(f'Counts are \n{class_counts}')
total_counts = np.sum(class_counts)
rates = class_counts / total_counts
print(f'Should sum to one: {np.sum(rates) == 1.0}')
print(f'\nRates are\n====================\n')
print(f'Vehicle: {rates[1]}')
print(f'Living: {rates[2]}')
print(f'Ground: {rates[3]}')
print(f'Manmade: {rates[4]}')
print(f'Vegetation: {rates[5]}')



inv_rates = 1-rates
# inv_rates[0] = 0
# inv_rates[8] = 0
# weights = np.round_((inv_rates) / np.sum(inv_rates), 20)

rates = rates[0:]
# weights = np.power(np.amax(rates) / rates, 1 / 3.0)
weights = 1 / rates

print(f'\nWeights are\n====================\n')
print(f'Ignore: {weights[0]}')
print(f'Vehicle: {weights[1]}')
print(f'Living: {weights[2]}')
print(f'Ground: {weights[3]}')
print(f'Manmade: {weights[4]}')
print(f'Vegetation: {weights[5]}')

print(f'\n ================= \n')
print(f'Weights sum to one: {(np.sum(weights) - 1.0) < 1e-3} ({np.sum(weights)})')

print(f'weights: {weights}')

Counts are 
[172123974, 1801972, 75612, 12203294, 4354036, 4257417]
Should sum to one: True

Rates are

Vehicle: 0.009249595407324864
Living: 0.0003881194646413194
Ground: 0.06264000336111497
Manmade: 0.02234944349242226
Vegetation: 0.021853494244231764

Weights are

Ignore: 1.131837131531718
Vehicle: 108.112836936423
Living: 2576.5262788975297
Ground: 15.964239245567631
Manmade: 44.74384341332961
Vegetation: 45.75927258241323


Weights sum to one: False (2792.238308206795)
weights: [   1.13183713  108.11283694 2576.5262789    15.96423925   44.74384341
   45.75927258]


In [13]:
mu = sem_cloud_features[:,0,:,:].mean()
sigma = sem_cloud_features[:,0,:,:].std()
print(f'We have a mean of {mu} and a std of {sigma}')

We have a mean of 1.5984423678560475 and a std of 8.400141767886875
